In [1]:
import numpy as np

from ase import Atoms, units
from ase.units import Angstrom
from ase.calculators.lj import LennardJones
from ase.optimize import BFGS
from ase.constraints import FixedPlane
from ase.visualize import view

from math import sqrt, exp

import matplotlib.pyplot as plt

## 3 Initial cluster setup
### 3.1 2D and 3D clusters with 7 atoms

In [2]:
def R(θ):
    """
    2D Rotation matrix for counterclockwise
    rotaion by θ around origin 
    """
    return np.array([[np.cos(θ), -np.sin(θ)]
                    ,[np.sin(θ),  np.cos(θ)]])

In [3]:
a = 1.1 * Angstrom
positions = np.zeros((7,3))
positions[1:,0] = a

rotations = np.array([np.linalg.matrix_power(R(np.pi/3),n) for n in range(6)])

positions[1:,:2] = np.einsum('ijk,ik->ij', rotations, positions[1:,:2])

In [4]:
cluster_2d = Atoms('7Ar',positions)
view(cluster_2d, viewer='x3d')

In [5]:
positions = np.zeros((7,3))
positions[[1,2],2] = a, -a
positions[3:,0] = a

rotations = np.array([np.linalg.matrix_power(R(np.pi/2),n) for n in range(4)])

positions[3:,:2] = np.einsum('ijk,ik->ij', rotations, positions[3:,:2])

In [6]:
cluster_3d = Atoms('7Ar',positions)
view(cluster_3d, viewer='x3d')

In [7]:
cluster_2d.calc = LennardJones()
cluster_3d.calc = LennardJones()

print('2D cluster:\npotential energy = %e [ε] \ninitial positions [Angstrom]:\n%s\n'
      %(cluster_2d.get_potential_energy(),cluster_2d.get_positions()))

print('3D cluster:\npotential energy = %e [ε] \ninitial positions [Angstrom]:\n%s\n'
      %(cluster_3d.get_potential_energy(),cluster_3d.get_positions()))

2D cluster:
potential energy = -1.228157e+01 [ε] 
initial positions [Angstrom]:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.10000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.50000000e-01  9.52627944e-01  0.00000000e+00]
 [-5.50000000e-01  9.52627944e-01  0.00000000e+00]
 [-1.10000000e+00  4.55774961e-16  0.00000000e+00]
 [-5.50000000e-01 -9.52627944e-01  0.00000000e+00]
 [ 5.50000000e-01 -9.52627944e-01  0.00000000e+00]]

3D cluster:
potential energy = -9.037942e+00 [ε] 
initial positions [Angstrom]:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.10000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.10000000e+00]
 [ 1.10000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.73555740e-17  1.10000000e+00  0.00000000e+00]
 [-1.10000000e+00  1.34711148e-16  0.00000000e+00]
 [-2.02066722e-16 -1.10000000e+00  0.00000000e+00]]



The default values of LJ parameters in ase are $\sigma=1$ and $\varepsilon=1$. Here $\sigma$ can be interpretten as the size of the particles and $\varepsilon$ is the potential depth.

### 3.2 Optimizing the initial cluster structures

In [8]:
for atom in cluster_2d:
    cluster_2d.set_constraint(FixedPlane(a=atom.index, direction=[0,0,1]))

cluster_2d_mini = BFGS(cluster_2d)
cluster_2d_mini.run(fmax=0.05)

cluster_3d_mini = BFGS(cluster_3d)
cluster_3d_mini.run(fmax=0.05)

      Step     Time          Energy         fmax
BFGS:    0 14:18:25      -12.281571        2.6446
BFGS:    1 14:18:25      -12.300262        1.9416
BFGS:    2 14:18:25      -12.415891        0.3876
BFGS:    3 14:18:25      -12.419639        0.0803
BFGS:    4 14:18:25      -12.419798        0.0025
      Step     Time          Energy         fmax
BFGS:    0 14:18:25       -9.037942        1.1509
BFGS:    1 14:18:25       -9.087128        0.2280
BFGS:    2 14:18:25       -9.088710        0.0317


True

In [9]:
print('2D cluster:\npotential energy = %e [ε] \npositions [Angstrom]:\n%s\n'
      %(cluster_2d.get_potential_energy(),cluster_2d.get_positions()))

print('3D cluster:\npotential energy = %e [ε] \npositions [Angstrom]:\n%s\n'
      %(cluster_3d.get_potential_energy(),cluster_3d.get_positions()))

2D cluster:
potential energy = -1.241980e+01 [ε] 
positions [Angstrom]:
[[-9.28125131e-16  8.75229936e-16 -4.68718085e-18]
 [ 1.11848102e+00 -1.86920528e-16 -1.42945281e-18]
 [ 5.59240510e-01  9.68632977e-01  7.90086046e-19]
 [-5.59240510e-01  9.68632977e-01  5.85319852e-19]
 [-1.11848102e+00  4.82495489e-16 -1.45987402e-19]
 [-5.59240510e-01 -9.68632977e-01  4.20336266e-19]
 [ 5.59240510e-01 -9.68632977e-01  0.00000000e+00]]

3D cluster:
potential energy = -9.088710e+00 [ε] 
positions [Angstrom]:
[[ 1.16567943e-17 -9.45012555e-18  4.98945265e-19]
 [-4.19308061e-18  3.13875808e-18  1.08627728e+00]
 [-3.50490677e-18  2.65483328e-18 -1.08627728e+00]
 [ 1.08627728e+00  1.32652411e-17 -1.34724577e-20]
 [ 6.22671853e-17  1.08627728e+00 -1.11484233e-19]
 [-1.08627728e+00  1.40575030e-16  1.02953425e-19]
 [-2.08905734e-16 -1.08627728e+00  0.00000000e+00]]



In [10]:
view(cluster_2d, viewer='x3d')

In [15]:
view(cluster_3d, viewer='x3d')

### 3.3 Pairwise distances

For a cluster with $N$ atoms there are $(N^2-N)/2$ pairwise distances without double counting.

## Setting a calculator, computing the energy, and minimization

In [12]:
# trimer along the x-axis
d = 1.0
trimer = Atoms('3Ar',[(0.0,0.0,0.0),(d,0.0,0.0),(-d,0.0,0.0)])

# getting the positions
pos = trimer.get_positions()

# modifying positions
pos += 0.1
        
# setting positions
trimer.set_positions(pos)

# getting all distances
alldist = trimer.get_all_distances()

# viewing the trimer
view(trimer)

In [13]:
# setting the calculator
ljcalc = LennardJones()
trimer.calc = ljcalc

# computing the energy
epot = trimer.get_potential_energy()

# constrain to xy-plane each atom index in trimer
trimer.set_constraint(FixedPlane(a=np.arange(len(trimer)), direction=[0,0,1]))

# setting and running a minimizer
trimer_mini = BFGS(trimer)
trimer_mini.run(fmax=0.05)

# Boltzman constant in units eV/K
units.kB

      Step     Time          Energy         fmax
BFGS:    0 14:18:25       -0.045085       23.8184
BFGS:    1 14:18:25       -1.786314        2.2635
BFGS:    2 14:18:25       -1.861534        2.0526
BFGS:    3 14:18:25       -0.617820       18.2380
BFGS:    4 14:18:25       -1.926487        1.7248
BFGS:    5 14:18:25       -1.967299        1.3685
BFGS:    6 14:18:25       -1.990990        1.3235
BFGS:    7 14:18:25       -2.012812        0.3185
BFGS:    8 14:18:25       -2.014631        0.0560
BFGS:    9 14:18:25       -2.014686        0.0031


8.617330337217213e-05

# Random numbers

In [14]:
from numpy.random import default_rng
# a random number generator
rng = default_rng(19884)

# a sample from the gaussian distribution N(0, 1)
x_g = rng.standard_normal()

# a sample from the uniform distribution on [0,1]
x_u = rng.uniform()